# Hydrometeorological time series

Notebook to create a .csv file with the hydrometeorological time series data for all CAMELS-DE catchments.  


In [1]:
import os
import pandas as pd
import geopandas as gpd

## Create folder structure

Inspired by CAMELS-UK

In [2]:
# create folder camels_de
os.makedirs("../output_data/camels_de", exist_ok=True)

# create subfolder timeseries
os.makedirs("../output_data/camels_de/timeseries", exist_ok=True)

## Trim the discharge data to 1951-2021

In [3]:
# read in all discharge data files
data_dir = "../../../discharge_harz"
# read all files to separate dfs and rename column according to the file name
# setup empty dict to store all dfs
dfs = {}
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        # read file with first column as index in format YYYY-MM-DD
        df = pd.read_csv(os.path.join(data_dir, file), index_col=0, parse_dates=True)
        df.columns = [file.split("_")[0]]
        # append df to dict
        dfs[file.split("_")[0]] = df

In [4]:
# check for duplicate / missing dates in individual dfs
for key, df in dfs.items():
    print(key)
    print(df.index.duplicated().sum())
    print(df.index.min(), df.index.max())
    print(df.index.freq)
    print(df.shape)

5
0
1979-11-01 00:00:00 2023-12-31 00:00:00
None
(16132, 1)
2
0
1948-11-01 00:00:00 2023-12-31 00:00:00
None
(27454, 1)
4
0
1931-11-01 00:00:00 2023-12-31 00:00:00
None
(33664, 1)
1
0
1963-11-01 00:00:00 2023-12-31 00:00:00
None
(21976, 1)
3
0
1938-07-01 00:00:00 2023-12-31 00:00:00
None
(26270, 1)


In [5]:
# combine all dfs to one df by joining on the index
df = pd.concat(dfs.values(), axis=1)
# trim the data to 1951-2021
df = df.loc["1951":"2021"]

### Check Q > 10 years after trimming to 1951-2020

In [6]:
# set threshold for minimum number of days / data points
threshold = 365*10

q_more_than_10_years = []

# get list of camels_ids
CAMELS_IDS = [1, 2, 3, 4, 5]

for camels_id in CAMELS_IDS:

    # get data from df according to camels_id
    data = df[[f"{camels_id}"]]

    # trim to start and end date
    start_date, end_date = "1951-01-01", "2020-12-31"
    data = data[(data.index >= start_date) & (data.index <= end_date)]

    # count q values that are not nan
    if f"{camels_id}" in data.columns:
        q_count = df[f"{camels_id}"].count()

        if q_count > threshold:
            q_more_than_10_years.append(camels_id)
            
print(f"Number of stations after threshold check: {len(q_more_than_10_years)}")

Number of stations after threshold check: 5


## Merge HYRAS with q data and save as .csv

For some stations, there are duplicated dates in the q data. We will remove these duplicates by taking the mean of the duplicates if the difference of the max and min q values to the mean q v/w value is smaller than 0.05.

In [8]:
# list of HYRAS variables
hyras_variables = ["Precipitation", "Humidity", "RadiationGlobal", "TemperatureMean", "TemperatureMin", "TemperatureMax"]

CATCHMENT_PATH = "../../../catchments_harz"
# Mapping catchment IDs to shapefiles
id_mapping = {
    1: dict(shapefile = "innerste_reservoir_catchment.shp", catchment_id = 1),
    2: dict(shapefile = "oker_reservoir_catchment.shp", catchment_id = 2),
    3: dict(shapefile = "ecker_reservoir_catchment.shp", catchment_id = 3),
    4: dict(shapefile = "soese_reservoir_catchment.shp", catchment_id = 4),
    5: dict(shapefile = "grane_reservoir_catchment.shp", catchment_id = 5),
}

for camels_id in CAMELS_IDS:

    # get data from df according to camels_id
    df_qw = df[[f"{camels_id}"]]
    
    # make sure date column is datetime
    df_qw.index = pd.to_datetime(df_qw.index)

    # rename columns q and w, if they exist
    if f"{camels_id}" in df_qw.columns:
        df_qw.rename(columns={f"{camels_id}": "discharge_vol"}, inplace=True)

    if "w" in df_qw.columns:
        df_qw.rename(columns={"w": "water_level"}, inplace=True)
    # else: add nan water_level column
    else:
        df_qw["water_level"] = None

    # calculate specific discharge with area from merit hydro
    # initialize station
    catchment_info = id_mapping[camels_id]
    shapefile_path = os.path.join(CATCHMENT_PATH, catchment_info["shapefile"])
    # get catchment
    catchment = gpd.read_file(shapefile_path)
    area = catchment.to_crs("EPSG:6933").area.values[0]  # in m^2
    df_qw["discharge_spec"] = ((df_qw["discharge_vol"] * 86400) / area) * 1000  # in mm/day

    # make index a column and rename it to date
    df_qw.reset_index(inplace=True)
    df_qw.rename(columns={"Date": "date"}, inplace=True)

    # reorder columns
    df_qw = df_qw[["date", "discharge_vol", "discharge_spec", "water_level"]]

    # read HYRAS data
    for variable in hyras_variables:
        # read hyras data
        df_hyras = pd.read_csv(os.path.join('..', '..', '..', 'camels_meteo', 'hyras', 'output_data', f'{camels_id}', 'data', f"{camels_id}_{variable}.csv"))

        # rename first column to date
        df_hyras.rename(columns={df_hyras.columns[0]: "date"}, inplace=True)

        # drop time from datetime date column in df_hyras
        df_hyras["date"] = pd.to_datetime(df_hyras["date"]).dt.date

        # make sure date columns are datetime before merging
        df_hyras["date"] = pd.to_datetime(df_hyras["date"])

        # merge data, keep all HYRAS data to make sure date range is correct
        df_qw = pd.merge(df_qw, df_hyras, on="date", how="right")

    # drop temperature columns
    df_qw.drop(columns=["tas_min", "tas_median", "tas_max", "tas_stdev",
                        "tasmin_min", "tasmin_median", "tasmin_max", "tasmin_stdev",
                        "tasmax_min", "tasmax_median", "tasmax_max", "tasmax_stdev"], inplace=True)
    
    # rename temperature columns
    df_qw.rename(columns={"tas_mean": "temperature_mean", "tasmin_mean": "temperature_min", "tasmax_mean": "temperature_max"}, inplace=True)
    # remame precipitation columns
    df_qw.rename(columns={"pr_mean": "precipitation_mean", "pr_min": "precipitation_min", "pr_median": "precipitation_median", "pr_max": "precipitation_max", "pr_stdev": "precipitation_stdev"}, inplace=True)
    # rename humidity columns 'hurs_mean', 'hurs_min', 'hurs_median', 'hurs_max', 'hurs_stdev'
    df_qw.rename(columns={"hurs_mean": "humidity_mean", "hurs_min": "humidity_min", "hurs_median": "humidity_median", "hurs_max": "humidity_max", "hurs_stdev": "humidity_stdev"}, inplace=True)
    # rename RadiationGlobal columns 'rsds_mean', 'rsds_min', 'rsds_median', 'rsds_max', 'rsds_stdev'
    df_qw.rename(columns={"rsds_mean": "radiation_global_mean", "rsds_min": "radiation_global_min", "rsds_median": "radiation_global_median", "rsds_max": "radiation_global_max", "rsds_stdev": "radiation_global_stdev"}, inplace=True)
    
    # round to 2 decimal places
    df_qw = df_qw.round(2)

    # sort columns
    df_qw = df_qw[["date", 
                   "discharge_vol", 
                   "discharge_spec", 
                   "water_level", 
                   "precipitation_mean",
                   "precipitation_min",
                   "precipitation_median", 
                   "precipitation_max",
                   "precipitation_stdev", 
                   "humidity_mean", 
                   "humidity_min",
                   "humidity_median",
                   "humidity_max", 
                   "humidity_stdev", 
                   "radiation_global_mean", 
                   "radiation_global_min", 
                   "radiation_global_median", 
                   "radiation_global_max", 
                   "radiation_global_stdev", 
                   "temperature_mean", 
                   "temperature_min", 
                   "temperature_max"]]

    # save to csv
    df_qw.to_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv", index=False)

print("Generated CAMELS-DE v1 hydrometeorological timeseries data!")

/tmp/ipykernel_812274/2607811390.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qw.rename(columns={f"{camels_id}": "discharge_vol"}, inplace=True)
/tmp/ipykernel_812274/2607811390.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qw["water_level"] = None
/tmp/ipykernel_812274/2607811390.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

Generated CAMELS-DE v1 hydrometeorological timeseries data!
